# Test if the notebook server is authorized to use Earth Engine

  * Click on the (potentially minimized) code block below and then press the 'Run' button to test if this server is authorized to talk to the Earth Engine backend servers.
  * If the server is authorized, you should see a map.
  * If the server is not authorized, you will see instructions for authorizing it.

Check if the IPython Widgets library is available on the server.

In [ ]:
try:
  import ipywidgets
  print('The IPython Widgets library (version {0}) is available on this server.'.format(
      ipywidgets.__version__
    ))
except ImportError:
  print('The IPython Widgets library is not available on this server.\n'
        'Please see https://github.com/jupyter-widgets/ipywidgets '
        'for information on installing the library.')
  raise

Check if the Earth Engine API is available on the server.

In [ ]:
try:
  import ee
  print('The Earth Engine Python API (version {0}) is available on this server.'.format(
      ee.__version__
    ))
except ImportError:
  print('The Earth Engine Python API library is not available on this server.\n'
        'Please see https://developers.google.com/earth-engine/python_install '
        'for information on installing the library.')
  raise

In [ ]:
import cStringIO
import os
import urllib

def handle_submit(sender):
  token = ee.oauth.request_token(get_auth_textbox.value.strip())
  ee.oauth.write_token(token)
  get_auth_response.value = 'Successfully saved authorization token.'
  get_auth_textbox.value = ''  # Clear the textbox.
  setAuthorizedStatus()

def revoke_credentials(sender):
  os.remove(ee.oauth.get_credentials_path())
  print('Credentials have been revoked.')
  setAuthorizedStatus()
  
def isAuthorized():
  try:
    ee.Initialize()
    return True
  except:
    return False

def setAuthorizedStatus():
  if isAuthorized():
    auth_status_button.description = 'The server is authorized to access Earth Engine'
    auth_status_button.button_style = 'success'
  else:
    auth_status_button.description = 'The server is not authorized to access Earth Engine'
    auth_status_button.button_style = 'danger'

# Define layouts for the table.
form_layout = ipywidgets.Layout(
    display='flex',
    flex_flow='column',
    border='solid',
    width='100%'
)
form_row_layout = ipywidgets.Layout(
    display='flex',
    flex_flow='row'
)
form_row_item_desc_layout = ipywidgets.Layout(
  width="100%"

)
form_row_item_status_layout = ipywidgets.Layout(
  width="100%",
  align_items='center'
)

# Define widgets that may be displayed.
auth_status_button = ipywidgets.Button(
  layout=form_row_item_status_layout
)
get_auth_textbox = ipywidgets.Text(
  placeholder='Paste authentication code here',
  description='Code:'
)
get_auth_textbox.on_submit(handle_submit)
get_auth_response = ipywidgets.Label('')

revoke_auth_button = ipywidgets.Button(
  layout=form_row_item_status_layout
)
revoke_auth_button.description = 'Click to revoke authorization'
revoke_auth_button.button_style = 'danger'
revoke_auth_button.on_click(revoke_credentials)

setAuthorizedStatus()

if isAuthorized():
  # Get a sample Earth Engine image.  
  url = ee.Image("CGIAR/SRTM90_V4").getThumbUrl({'min':0, 'max':3000})
  auth_instructions = ipywidgets.VBox(
    [ipywidgets.Image(
      value=cStringIO.StringIO(urllib.urlopen(url).read()).read(),
      format='png',
      width=400,
      height=200
    ),
    revoke_auth_button
    ],
    layout=form_row_item_status_layout
  )
else:
  auth_instructions = ipywidgets.VBox([
      ipywidgets.HTML(
        'Click on the link below to start the authentication and authorization process. '
        'Once you have received an authentication code, paste it in the box below and press return.'
      ),
      ipywidgets.HTML(
        '<a href="{url}" target="auth">Open Authentication Tab</a><br/>'.format(
          url=ee.oauth.get_authorization_url()
        )
      ),
      get_auth_textbox,
      get_auth_response
    ],
    layout=form_row_item_status_layout
  )

form = ipywidgets.Box(
  [auth_status_button, auth_instructions],
  layout=form_layout
)
form